In [1]:
from uncertainty_engine import Client
from pprint import pprint

In [2]:
client = Client(
    env="dev"
)
client.authenticate()

In [3]:
add_node_info = client.get_node_info("Add")

In [4]:
add_node_info.inputs

{'lhs': NodeInputInfo(type='float', label='LHS', description='Left-hand side of the addition', required=True, set_in_node=True, default=None),
 'rhs': NodeInputInfo(type='float', label='RHS', description='Right-hand side of the addition', required=True, set_in_node=True, default=None)}

In [5]:
add_node_info.outputs

{'ans': NodeOutputInfo(type='float', label='Answer', description='Result of the addition')}

Okay so can use `get_node_info` to get the required information of the node in 
order to save the relevent information about the input to the `tool_metadata`

In [6]:
from uncertainty_engine.nodes.basic import Add
from uncertainty_engine.nodes.base import Node

In [7]:
add_number_info = client.get_node_info("Number")

Number node isn't really a SDK thing but useful for this example?

In [8]:
add_number_info

NodeInfo(id='Number', label='Number', category='Input', description='Define a number', long_description='Define any decimal value, like "3.14" or "4"', image_name='uncertainty-engine-number-node', cost=1, inputs={'value': NodeInputInfo(type='float', label='Number', description='Real number', required=True, set_in_node=True, default=None)}, outputs={'value': NodeOutputInfo(type='float', label='Number', description='Real number')}, requirements=NodeRequirementsInfo(cpu=256, gpu=False, memory=512, timeout=15), load_balancer_url=None, queue_url='https://sqs.eu-west-2.amazonaws.com/939027885851/dev-uncertainty-engine-number-nod-uncertaintyenginenumbernodeQueueD-k7SMxVgEYvOu', cache_url='redis-13325.c338.eu-west-2-1.ec2.redns.redis-cloud.com', version_types_lib='0.9.0', version_base_image=2, version_node=2, service_arn='arn:aws:ecs:eu-west-2:939027885851:service/ue_dev/uncertainty-engine-number-node')

In [9]:
number_value_input = add_number_info.inputs

In [10]:
number = Node(
    node_name="Number",
    label="num node",
    value="5",
)

number.add_tool_input("value", add_number_info)


number_handle = number.make_handle("value")

add = Add(lhs = 4,
          rhs = number_handle,
          label="add node")

add.add_tool_input("lhs", add_node_info)
add.add_tool_input("rhs", add_node_info)
add.add_tool_output("ans", add_node_info)

output_result = add.make_handle("ans")


In [11]:
print(number)

In [13]:
number.__dict__

{'node_name': 'Number',
 'label': 'num node',
 'tool_metadata': {'tool_inputs': {'value': NodeInputInfo(type='float', label='Number', description='Real number', required=True, set_in_node=True, default=None)}},
 'value': '5'}

In [14]:
add.__dict__

{'node_name': 'Add',
 'label': 'add node',
 'tool_metadata': {'tool_inputs': {'lhs': NodeInputInfo(type='float', label='LHS', description='Left-hand side of the addition', required=True, set_in_node=True, default=None),
   'rhs': NodeInputInfo(type='float', label='RHS', description='Right-hand side of the addition', required=True, set_in_node=True, default=None)},
  'tool_outputs': {'ans': NodeOutputInfo(type='float', label='Answer', description='Result of the addition')}},
 'lhs': 4,
 'rhs': Handle(node_name='num node', node_handle='value')}

In [15]:
from uncertainty_engine.graph import Graph

# Create a new graph
add_graph = Graph()

In [ ]:
add_graph.add_node(number)

add_graph.add_node(add)


Processing metadata
Processing metadata


In [17]:
pprint(add_graph.nodes)

{'nodes': {'add node': {'inputs': {'lhs': {'node_handle': 'add node_lhs',
                                           'node_name': '_'},
                                   'rhs': {'node_handle': 'value',
                                           'node_name': 'num node'},
                                   'tool_metadata': {'node_handle': 'add '
                                                                    'node_tool_metadata',
                                                     'node_name': '_'}},
                        'type': 'Add'},
           'num node': {'inputs': {'tool_metadata': {'node_handle': 'num '
                                                                    'node_tool_metadata',
                                                     'node_name': '_'},
                                   'value': {'node_handle': 'num node_value',
                                             'node_name': '_'}},
                        'type': 'Number'}}}


In [18]:
pprint(add_graph.tool_metadata)

{'inputs': {'add node': {'lhs': NodeInputInfo(type='float', label='LHS', description='Left-hand side of the addition', required=True, set_in_node=True, default=None),
                         'rhs': NodeInputInfo(type='float', label='RHS', description='Right-hand side of the addition', required=True, set_in_node=True, default=None)},
            'num node': {'value': NodeInputInfo(type='float', label='Number', description='Real number', required=True, set_in_node=True, default=None)}},
 'outputs': {'add node': {'ans': NodeOutputInfo(type='float', label='Answer', description='Result of the addition')}}}


In [19]:
from uncertainty_engine.nodes.workflow import Workflow

In [20]:
add_workflow = Workflow(
    graph=add_graph.nodes,
    inputs=add_graph.external_input,
    external_input_id=add_graph.external_input_id,
    requested_output={
        "add result": output_result.model_dump(),
        }
    )

In [ ]:
# TODO need to dump
add_response = client.run_node(add_workflow)

TypeError: Object of type NodeInputInfo is not JSON serializable

In [74]:
pprint(add_response.model_dump()["inputs"])

{'external_input_id': '_',
 'graph': {'nodes': {'add node': {'inputs': {'lhs': {'node_handle': 'add '
                                                                    'node_lhs',
                                                     'node_name': '_'},
                                             'rhs': {'node_handle': 'value',
                                                     'node_name': 'num node'}},
                                  'type': 'Add'},
                     'num node': {'inputs': {'value': {'node_handle': 'num '
                                                                      'node_value',
                                                       'node_name': '_'}},
                                  'type': 'Number'}}},
 'inputs': {'add node_lhs': 4, 'num node_value': '5'},
 'requested_output': {'add result': {'node_handle': 'ans',
                                     'node_name': 'add node'}}}


In [75]:
pprint(add_response.model_dump())

{'inputs': {'external_input_id': '_',
            'graph': {'nodes': {'add node': {'inputs': {'lhs': {'node_handle': 'add '
                                                                               'node_lhs',
                                                                'node_name': '_'},
                                                        'rhs': {'node_handle': 'value',
                                                                'node_name': 'num '
                                                                             'node'}},
                                             'type': 'Add'},
                                'num node': {'inputs': {'value': {'node_handle': 'num '
                                                                                 'node_value',
                                                                  'node_name': '_'}},
                                             'type': 'Number'}}},
            'inputs': {'add node_lhs': 4, 'num 